# 06 Crowding Overlay


このノートブックは `config/config.example.yaml` を読み込み、synthetic データで手法を単体検証します。

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "config" / "config.example.yaml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import load_config
from src.utils.calendar import month_end_dates, quarter_end_dates
from src.utils.io import save_parquet, save_csv, save_figure
from src.utils.validation import assert_weights_sum_to_one, assert_no_lookahead

cfg = load_config(PROJECT_ROOT / "config" / "config.example.yaml")
np.random.seed(cfg["project"]["seed"])

for rel in ["outputs/logs", "outputs/signals", "outputs/weights", "outputs/diagnostics", "outputs/figures"]:
    (PROJECT_ROOT / rel).mkdir(parents=True, exist_ok=True)

def make_synthetic(cfg):
    dates = pd.bdate_range(cfg["data"]["date_start"], cfg["data"]["date_end"])
    themes = cfg["data"]["assets"]["theme_ids"]
    n, m = len(dates), len(themes)

    factors = np.random.normal(0.0, 0.004, size=(n, 3))
    loadings = np.random.uniform(-1.0, 1.0, size=(m, 3))
    eps = np.random.normal(0.0, 0.01, size=(n, m))
    returns = pd.DataFrame(factors @ loadings.T + eps, index=dates, columns=themes)
    prices = 100.0 * (1.0 + returns).cumprod()

    flow_common = np.random.normal(0.0, 1.0, size=n)
    flow_noise = np.random.normal(0.0, 0.6, size=(n, m))
    flow = pd.DataFrame(flow_common[:, None] + flow_noise, index=dates, columns=themes)

    theme_scores = flow.rolling(21, min_periods=5).mean() + np.random.normal(0.0, 0.2, size=(n, m))
    stale_cols = themes[2::2]
    if len(stale_cols) > 0:
        theme_scores.loc[dates[-120:], stale_cols] = np.nan

    if cfg["calendar"]["rebalance"] == "Q":
        rebalance_dates = quarter_end_dates(dates)
    else:
        rebalance_dates = month_end_dates(dates)

    return dates, themes, returns, prices, flow, theme_scores, rebalance_dates


In [ ]:
from src.risk.crowding import compute_crowding_components, aggregate_crowding, apply_crowding_overlay

dates, themes, returns, prices, flow, theme_scores, rebalance_dates = make_synthetic(cfg)

rebal = pd.DataFrame(0.0, index=rebalance_dates, columns=themes)
for d in rebalance_dates:
    rebal.loc[d] = 1.0 / len(themes)
base_weights = rebal.reindex(dates).ffill().fillna(1.0 / len(themes))

components = compute_crowding_components(
    prices=prices,
    calendar=cfg["calendar"],
    window=cfg["crowding"]["window_days"],
)
score = aggregate_crowding(components, method="mean", handle_missing="renormalize")
overlay_weights = apply_crowding_overlay(
    base_weights=base_weights,
    crowding_score=score,
    cap_quantile=cfg["crowding"]["overlay"]["cap_quantile"],
    scale=cfg["crowding"]["overlay"]["scale"],
)

assert_weights_sum_to_one(overlay_weights)
weights_applied = overlay_weights.shift(1)
assert weights_applied.iloc[0].isna().all()

save_parquet(components, PROJECT_ROOT / "outputs/signals/06_crowding_components.parquet")
save_parquet(score, PROJECT_ROOT / "outputs/signals/06_crowding_score.parquet")
save_parquet(overlay_weights, PROJECT_ROOT / "outputs/weights/06_overlay_weights.parquet")

comp_diag = components.copy()
if isinstance(comp_diag.columns, pd.MultiIndex):
    comp_diag.columns = [f"{a}__{b}" for a, b in comp_diag.columns]
save_csv(comp_diag, PROJECT_ROOT / "outputs/diagnostics/06_crowding_components.csv")

fig, ax = plt.subplots(figsize=(10, 4))
score.mean(axis=1).plot(ax=ax)
ax.set_title("06 Crowding Score (Cross-Section Mean)")
ax.set_ylabel("score")
save_figure(fig, PROJECT_ROOT / "outputs/figures/06_crowding_timeseries.png")
plt.close(fig)

print(components.shape, score.shape, overlay_weights.shape)


In [ ]:
required = [
    "outputs/signals/06_crowding_components.parquet",
    "outputs/signals/06_crowding_score.parquet",
    "outputs/weights/06_overlay_weights.parquet",
    "outputs/diagnostics/06_crowding_components.csv",
    "outputs/figures/06_crowding_timeseries.png",
]
status = "OK" if all((PROJECT_ROOT / p).exists() for p in required) else "NG"
checks = ", ".join([f"{p}:{(PROJECT_ROOT / p).exists()}" for p in required])
line = f"Notebook=06_crowding_overlay.ipynb, {status}, 生成ファイル確認={checks}\n"
with (PROJECT_ROOT / "outputs/logs/qa_checklist.txt").open("a", encoding="utf-8") as f:
    f.write(line)
print(line)
